This file is for the assignment 6 of Monte Carlo Method at University Paris Dauphine master 2 MASEF

Author: Yu Xiang

Contact: shawnxiangyu@yahoo.com



In [60]:
# import libaries and set the setting for plot
%matplotlib inline
import numpy as np

import time
import matplotlib.pyplot as plt
from scipy.stats import norm
import math
from scipy.stats.stats import pearsonr 
import pandas as pd
import math
from scipy import stats 
norm_ppf = stats.norm.ppf
# choose a large font size by default and use tex for math
fontsize = 10
params = {'axes.labelsize': fontsize + 2,
      'font.size': fontsize + 2,
      'legend.fontsize': fontsize + 2,
      'xtick.labelsize': fontsize,
      'ytick.labelsize': fontsize}
plt.rcParams.update(params)

In [61]:
# parameters
r = 0.0
sigma = 0.25 
X0 = 100
T = 1 
ninty5_quantile = norm_ppf(0.975)
Ks = [50, 100, 150]
nr_ks = len(Ks)

In [62]:
# Pre-calculation: Black Scholes formula for European call

def bs_euro_call(St, t, K): 
    rt = T - t  # remaining time to maturity
    d1 = 1/ (sigma * rt ** 0.5) * (np.log(St/K) + (r + sigma ** 2 / 2) * rt)
    d2 = d1 - sigma * rt ** 0.5 
    PV_K = K * np.exp(-r * rt)
    
    C_St_t = norm.cdf(d1) * St - norm.cdf(d2) * PV_K
    
    return C_St_t

op_values = np.zeros(nr_ks)
dict_K_V = {}

for ki in range(nr_ks): 
    K = Ks[ki]
    op_values[ki] = bs_euro_call(X0, 0, K)
    dict_K_V[str(K)] = op_values[ki]    
    print ('\nThe theoretical value of the European call option with strike price K = ' + str(K), ' is :', op_values[ki])




The theoretical value of the European call option with strike price K = 50  is : 50.01464853081623

The theoretical value of the European call option with strike price K = 100  is : 9.94764496602258

The theoretical value of the European call option with strike price K = 150  is : 0.6718625762008115


# Exercise 1: Importance Samping

## Exercise 1.1 Simple Monte Carlo simulation

In [63]:
# Compute the European call via Monte Carlo Method Based on Black Scholes model
def sim_XT_WT(T=1, X0=100, r=0, sigma=0.25, nr_sim=10000):  
    # this function generates the simulated XT and WT at the end of the valuation period 
    WT = T ** 0.5 * np.random.randn(nr_sim, 1)
    XT = X0 * np.exp((r - sigma ** 2 / 2) * T + sigma * WT) 
    return XT, WT


In [64]:
# 1.1 Compute the price of European call option with Srike K and Maturity T 
Ks = [50, 100, 150]
nr_sims = [10000, 100000]
nr_cases = len(nr_sims) * len(Ks)

conf_int = np.zeros((nr_cases, 7))
i = 0

for si in range(len(nr_sims)): 
    nr_simi = nr_sims[si]
    XT, WT = sim_XT_WT(nr_sim=nr_simi)
    
    for ki in range(len(Ks)):

        K = Ks[ki]
        
        X = np.exp(- r * T) * np.maximum(XT - K, 0)
        EX = np.mean(X) 
        var_n = nr_simi /(nr_simi - 1) * (np.mean(X ** 2) - EX ** 2) 
        half_int = ninty5_quantile * var_n ** 0.5 / nr_simi ** 0.5
        conf_interval = np.round([EX - half_int,  EX + half_int],4)
        
        conf_int[i, 0] = nr_simi
        conf_int[i, 1] = K
        conf_int[i, 2] = dict_K_V[str(K)]
        conf_int[i, 3] = EX
        conf_int[i, 4:6] = conf_interval
        conf_int[i, 6] = half_int * 2
        i += 1
        
Simple_MC = conf_int.copy()

In [65]:
# normal simulation result
import pandas as pd
result = pd.DataFrame(Simple_MC,
                      columns=['Nr_sims', 'K', 'theoretical_price',  'numerical_price', 
                               'lower_bound', 'upper_bound','int_len'])

result

,Nr_sims,K,theoretical_price,numerical_price,lower_bound,upper_bound,int_len
0,10000.0,50.0,50.014649,49.772252,49.2774,50.2671,0.989766
1,10000.0,100.0,9.947645,9.696696,9.3640,10.0294,0.665314
2,10000.0,150.0,0.671863,0.721373,0.6249,0.8178,0.192853
3,100000.0,50.0,50.014649,50.056599,49.8991,50.2141,0.315089
4,100000.0,100.0,9.947645,9.978977,9.8736,10.0844,0.210814
5,100000.0,150.0,0.671863,0.676548,0.6477,0.7054,0.057649


### 1.1 Comment: 
With more simulations, variance becomes smaller. When the option is deep in the money, the value of the option is almost a linear relation with the strike price.  However when the option is deep out of money, the value of the option will be very small. 

## 1.2 Importance Sampling

In [66]:
# simulate XT^h and WT based on Black scholes formula
def sim_XTh_WT(T=1, X0=100, r=0, h=0, sigma=0.25, nr_sim=10000):  
    # this function generates the simulated XT and WT at the end of the valuation period 
    
    WT = T ** 0.5 * np.random.randn(nr_sim, 1)
    XTh = X0 * np.exp((r + sigma * h - sigma ** 2 / 2) * T + sigma * WT) 
    return XTh, WT

In [67]:
# 1.1 Compute the price of European call option with Srike K and Maturity T 
Ks = [50, 100, 150]
nr_sims = [10000, 100000]
nr_cases = len(nr_sims) * len(Ks)

conf_int = np.zeros((nr_cases, 7))
i = 0

for si in range(len(nr_sims)): 
    nr_simi = nr_sims[si]
    for ki in range(len(Ks)):
        K = Ks[ki]
        if K >= X0: 
            h = 2
        else:     
            h = 0.5
        XTh, WT = sim_XTh_WT(nr_sim=nr_simi, h=h)
        X = np.exp(- r * T) * np.exp(- h * WT - h ** 2 * T / 2) * np.maximum(XTh - K, 0)
        EX = np.mean(X) 
        var_n = nr_simi /(nr_simi - 1) * (np.mean(X ** 2) - EX ** 2) 
        half_int = ninty5_quantile * var_n ** 0.5 / nr_simi ** 0.5
        conf_interval = np.round([EX - half_int,  EX + half_int],4)
        
        conf_int[i, 0] = nr_simi
        conf_int[i, 1] = K
        conf_int[i, 2] = dict_K_V[str(K)]
        conf_int[i, 3] = EX
        conf_int[i, 4:6] = conf_interval
        conf_int[i, 6] = half_int * 2
        i += 1
        
IS_MC = conf_int.copy()

In [68]:
# Importance sampling simulation result
import pandas as pd
result = pd.DataFrame(IS_MC,
                      columns=['Nr_sims', 'K', 'theoretical_price',  'numerical_price', 
                               'lower_bound', 'upper_bound','int_len'])

result

,Nr_sims,K,theoretical_price,numerical_price,lower_bound,upper_bound,int_len
0,10000.0,50.0,50.014649,50.034687,49.9457,50.1236,0.177916
1,10000.0,100.0,9.947645,9.972411,9.8043,10.1405,0.336128
2,10000.0,150.0,0.671863,0.665407,0.6526,0.6782,0.025641
3,100000.0,50.0,50.014649,49.990086,49.9613,50.0189,0.057619
4,100000.0,100.0,9.947645,9.921218,9.8680,9.9744,0.106359
5,100000.0,150.0,0.671863,0.673788,0.6697,0.6779,0.008149


In [69]:
# 1.2 Compare the result
conf_inter_diff = Simple_MC[:,6] - IS_MC[:,6]
conf_inter_times = Simple_MC[:,6] / IS_MC[:,6]
Compare = np.hstack((Simple_MC, IS_MC[:,3:], conf_inter_diff[:,np.newaxis], conf_inter_times[:,np.newaxis]))


# display summary and compare
import pandas as pd
def highlight_cols(s):
    color = 'grey'
    return 'background-color: %s' % color

Exercise_1_result = pd.DataFrame(Compare,
                      columns=['Nr_sim','K','theoretical_price', 'S_MC_price',
                               'S_MC_lower', 'S_MC_upper', 'S_MC_int_len','IS_MC_price',
                               'IS_MC_lower', 'IS_MC_upper', 'IS_MC_int_len', 'diff_intlen', 'variance_ratio'])


Exercise_1_result.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['S_MC_int_len', 'IS_MC_int_len', 'variance_ratio']])

,Nr_sim,K,theoretical_price,S_MC_price,S_MC_lower,S_MC_upper,S_MC_int_len,IS_MC_price,IS_MC_lower,IS_MC_upper,IS_MC_int_len,diff_intlen,variance_ratio
0,10000,50,50.0146,49.7723,49.2774,50.2671,0.989766,50.0347,49.9457,50.1236,0.177916,0.811851,5.56312
1,10000,100,9.94764,9.6967,9.364,10.0294,0.665314,9.97241,9.8043,10.1405,0.336128,0.329186,1.97935
2,10000,150,0.671863,0.721373,0.6249,0.8178,0.192853,0.665407,0.6526,0.6782,0.0256407,0.167212,7.52136
3,100000,50,50.0146,50.0566,49.8991,50.2141,0.315089,49.9901,49.9613,50.0189,0.0576192,0.25747,5.46848
4,100000,100,9.94764,9.97898,9.8736,10.0844,0.210814,9.92122,9.868,9.9744,0.106359,0.104454,1.98209
5,100000,150,0.671863,0.676548,0.6477,0.7054,0.0576492,0.673788,0.6697,0.6779,0.00814856,0.0495007,7.07478


## 1.2 Comment: 
The choice of $h$ is an art. Morever, when $h$ is proper chosen, we do observe significantly reduction of the variance with importance sampling method. 

Optional value at time $t_0$ is: 
$$V_0 = E[e^{-rT}(X_T - K)^+] = e^{-rT}*E[(X_T - K)^+] = e^{-rT}*E[G(W_T)]$$

Where, 
$$G(W_T) = (X_T - K)^+ = \{X_0 e^{(r - \frac{\sigma ^2}{2})T + \sigma W_T} - K\}^+$$


In [70]:
def G(WT, X0=100, r=0.0, T=1, sigam=0.5, K=100): 
    return np.maximum(X0 * np.exp((r- sigma ** 2 / 2) * T + sigma * WT) - K, 0)


def F(ht, Ztp, Gzt): 
    
    var_term =  Gzt ** 2 * np.exp(-ht * Ztp + ht ** 2 / 2)
    return (ht - Ztp) * var_term, var_term


## 1.3 Implement the Robbins-Monro algorithm


In [71]:
Ks = [50, 100, 150]
nr_sims = [10000, 100000]
nr_cases = len(nr_sims) * len(Ks)

conf_int = np.zeros((nr_cases, 7))

h0 = 1 # initilize h0 as 
U0 = 1
count = 0
i = 0
printint = [1, 2, 300,301]
for si in range(len(nr_sims)): 
    nr_simi = nr_sims[si]
    for ki in range(len(Ks)):
        K = Ks[ki]
        
        #XT, WT = sim_XT_WT(nr_sim=nr_simi)
        # here we find the optimal h: using Robbins-Monro algorithm 
        h_pre = h0
        
        nr_RM_iterations = nr_simi
        
        Gts = np.zeros(nr_RM_iterations)
        ys = np.zeros(nr_RM_iterations)
        hts = np.zeros(nr_RM_iterations)
        var_terms = np.zeros(nr_RM_iterations) 
        for iter in range(nr_RM_iterations):
            
            gamma = 1 / (iter + 1)
            zt = T ** 0.5 * np.random.randn(1)[0]
            Gzt = G(WT=zt, K=K) 
            Yt_plus, var_term = F(h_pre, zt, Gzt)
            
            update = h_pre - gamma * Yt_plus
            var_terms[iter] = var_term 
                 
            if np.abs(update) < (np.log(iter+1) / 6) ** 0.5 + U0:                 
                ht_plus = h_pre - gamma * Yt_plus
            else: 
                ht_plus = h_pre        
            h_pre = ht_plus   
               
            Gts[iter] = Gzt  
            if iter == 0: 
                ys[iter] = Gzt
                
            else: 
                ys[iter] = iter / (iter + 1) * ys[iter - 1] + 1 / (iter + 1) * Gzt  
                
            hts[iter] = h_pre               
        # print ('\n nr_sims=' + str(nr_simi), 'K=' + str(K), 'hopt: ' + str(hts[-1]), 'E: ' + str(ys[-1]), 'F:' + str( F(hts[-1], zt, Gzt))) 
    

        EX = ys[iter-1] 
        vy = iter ** 0.5 * (ys - EX)
        #var_n = nr_simi /(nr_simi - 1) * (np.mean(vy ** 2) - np.mean(vy) ** 2) 
        var_n = np.mean(var_terms)
        half_int = ninty5_quantile * var_n ** 0.5 / nr_simi ** 0.5
        conf_interval = np.round([EX - half_int,  EX + half_int],4)
        
        conf_int[i, 0] = nr_simi
        conf_int[i, 1] = K
        conf_int[i, 2] = dict_K_V[str(K)]
        conf_int[i, 3] = EX
        conf_int[i, 4:6] = conf_interval
        conf_int[i, 6] = half_int * 2
        i += 1
        
RM_MC = conf_int.copy()        

In [72]:
# hts[:100]

In [73]:
# Importance sampling simulation result
import pandas as pd
result = pd.DataFrame(RM_MC,
                      columns=['Nr_sims', 'K', 'theoretical_price',  'numerical_price', 
                               'lower_bound', 'upper_bound','int_len'])

result

,Nr_sims,K,theoretical_price,numerical_price,lower_bound,upper_bound,int_len
0,10000.0,50.0,50.014649,49.904139,46.9470,52.8613,5.914337
1,10000.0,100.0,9.947645,9.868142,9.6436,10.0927,0.449021
2,10000.0,150.0,0.671863,0.721230,0.7023,0.7402,0.037910
3,100000.0,50.0,50.014649,49.889530,49.5136,50.2655,0.751852
4,100000.0,100.0,9.947645,9.877596,9.8075,9.9477,0.140260
5,100000.0,150.0,0.671863,0.678762,0.6732,0.6844,0.011214


In [74]:
# 1.2 Compare the result
conf_inter_diff = Simple_MC[:,6] - RM_MC[:,6]
conf_inter_times = Simple_MC[:,6] / RM_MC[:,6]
Compare = np.hstack((Simple_MC, RM_MC[:,3:], conf_inter_diff[:,np.newaxis], conf_inter_times[:,np.newaxis]))


# display summary and compare
import pandas as pd
def highlight_cols(s):
    color = 'grey'
    return 'background-color: %s' % color

Exercise_1_result = pd.DataFrame(Compare,
                      columns=['Nr_sim','K','theoretical_price', 'S_MC_price',
                               'S_MC_lower', 'S_MC_upper', 'S_MC_int_len','RM_MC_price',
                               'RM_MC_lower', 'RM_MC_upper', 'RM_MC_int_len', 'diff_intlen', 'variance_ratio'])


Exercise_1_result.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['S_MC_int_len', 'RM_MC_int_len', 'variance_ratio']])

,Nr_sim,K,theoretical_price,S_MC_price,S_MC_lower,S_MC_upper,S_MC_int_len,RM_MC_price,RM_MC_lower,RM_MC_upper,RM_MC_int_len,diff_intlen,variance_ratio
0,10000,50,50.0146,49.7723,49.2774,50.2671,0.989766,49.9041,46.947,52.8613,5.91434,-4.92457,0.16735
1,10000,100,9.94764,9.6967,9.364,10.0294,0.665314,9.86814,9.6436,10.0927,0.449021,0.216293,1.4817
2,10000,150,0.671863,0.721373,0.6249,0.8178,0.192853,0.72123,0.7023,0.7402,0.0379097,0.154943,5.08715
3,100000,50,50.0146,50.0566,49.8991,50.2141,0.315089,49.8895,49.5136,50.2655,0.751852,-0.436763,0.419084
4,100000,100,9.94764,9.97898,9.8736,10.0844,0.210814,9.8776,9.8075,9.9477,0.14026,0.0705537,1.50302
5,100000,150,0.671863,0.676548,0.6477,0.7054,0.0576492,0.678762,0.6732,0.6844,0.0112143,0.0464349,5.14069


# Question remaining to be answered? 
- which initial $h_0$ should we taken?
- how to calculate the variance?
 

# Exercise 2: Sensitivities

$$
X_T = X_0 e^{(r- \frac{\sigma ^2}{2})T + \sigma W_T }
$$

$$P(X_T \in [a, b]) = P(X_T <=b) - P(X_T <=a)$$

Where
$$
P(X_T <=b) = P(\log X_T <=\log b) = P(\log (X_0 e^{(r- \frac{\sigma ^2}{2})T} * e^{\sigma W_T }) <=\log b) \\
=  P(\log (X_0 e^{(r- \frac{\sigma ^2}{2})T}) + \sigma \sqrt T * Z_T  <=\log b) \\
= P(\sigma \sqrt T * Z_T  <=\log b - \log (X_0 e^{(r- \frac{\sigma ^2}{2})T})) \\
= P( Z_T  <= \frac{\log b - \log (X_0 e^{(r- \frac{\sigma ^2}{2})T})}{\sigma \sqrt T})
$$


In [75]:
# 2.0 Parameters and pre-computation

a = 95
b = 105
nr_sim = 50000

XT, WT = sim_XT_WT(nr_sim=nr_sim)
in_id = np.logical_and(XT>=a, XT<=b)
value = np.sum(in_id) /len(in_id) # this is the expecation

var_n = nr_sim /(nr_sim - 1) * (np.mean(in_id ** 2) - np.mean(in_id) ** 2) 

half_int = ninty5_quantile * var_n ** 0.5 / nr_sim ** 0.5
conf_interval = np.round([value - half_int,  value + half_int], 5)

print ('value and confidence interval are : ', value, ' ',  conf_interval)

value and confidence interval are :  0.1558   [0.15262 0.15898]


In [76]:
zb = (np.log(b) - np.log(X0 * np.exp(r - sigma ** 2 / 2) * T)) / (sigma * T ** 0.5)
za = (np.log(a) - np.log(X0 * np.exp(r - sigma ** 2 / 2) * T)) / (sigma * T ** 0.5)
from scipy.stats import norm
xb = norm.cdf(zb)
xa = norm.cdf(za)
value = xb -xa
print ('The real value of this option is : ', value)

The real value of this option is :  0.15752696559852564


## 2.1 Compute the Gamma using a finite difference schme 

$$\Gamma = \frac{\partial \Delta}{\partial S} = \frac{\partial^2 V}{\partial S^2}$$

Gamma: $\Gamma$, measures the rate of change in the delta with respect to changes in the underlying price. Gamma is the second derivative of the value function with respect to the underlying price. 

Delta: $\Delta$, measures the rate of change of the theoretical option value with respect to changes in the underlying asset's price. Delta is the partial derivative|first derivative of the value $V$ of the option with respect to the underlying instrument's price $S$.
$$\Delta = \frac{\partial V}{\partial S}$$




In [85]:
# Compute the European call via Monte Carlo Method Based on Black Scholes model


In [92]:
epsilons = [0.5, 0.1, 0.05, 0.005]

# a = 0.95
# b = 1.05
# X0 = 1
nr_sim = 50000
nr_eps = len(epsilons)
deltas = np.zeros(nr_eps)
gammas = np.zeros((nr_eps, 5))

for i in range(nr_eps): 
    epsilon_i = epsilons[i]

     XT, WT = sim_XT_WT(nr_sim=nr_sim, X0=X0)
#     X0p = X0 + 2 *  epsilon_i
#     X0m = X0 - 2 * epsilon_i
    
     X0p = X0 +  epsilon_i
     X0m = X0 -  epsilon_i
    
    
#     XT, XTp, XTm = sim_XT(nr_sim=nr_sim, epsilon=2 * epsilon_i)
    

    XTm, WTm = sim_XT_WT(nr_sim=nr_sim, X0=X0m)
    
    in_id = np.logical_and(XT>=a, XT<=b)
    value = np.sum(in_id) /len(in_id)
    var = np.mean(in_id ** 2) - np.mean(in_id) ** 2
    
    in_id = np.logical_and(XTp>=a, XTp<=b)
    pvalue = np.sum(in_id) /len(in_id)
    varp = np.mean(in_id ** 2) - np.mean(in_id) ** 2
    
    delta_p = (pvalue - value) /  (2 * epsilon_i)
    
    XT, XTp, XTm = sim_XT(nr_sim=nr_sim, epsilon=2 * epsilon_i)
    
    in_id = np.logical_and(XT>=a, XT<=b)
    value = np.sum(in_id) /len(in_id)
    var = np.mean(in_id ** 2) - np.mean(in_id) ** 2
    
    
    
    

    in_id = np.logical_and(XTm>=a, XTm<=b)
    mvalue = np.sum(in_id) / len(in_id)
    varm = np.mean(in_id ** 2) - np.mean(in_id) ** 2
    
    var_n = (varp + varm + 4 * var)/ (epsilon_i ** 4)
    
    half_int = ninty5_quantile * var_n ** 0.5 / nr_sim ** 0.5
    
    # delta_p = (pvalue - value) /  (2 * epsilon_i)
    delta_m = (value - mvalue) /  (2 * epsilon_i) 
    gamma = (delta_p - delta_m) / (2 * epsilon_i)
    #gamma = (pvalue - 2 * value + mvalue) / (epsilon_i ** 2 )
    conf_interval = np.round([gamma - half_int,  gamma + half_int], 5)
    
    gammas[i, 0] = epsilon_i
    gammas[i, 1] = gamma  
    gammas[i, 2:4] = conf_interval
    gammas[i, 4] = half_int * 2 
    # deltas[i] = (delta_p + delta_m) / 2
    # gammas[i] = (delta_p - delta_m) /  epsilon_i
    

np.set_printoptions(suppress=True)
import pandas as pd
result = pd.DataFrame(gammas,
                      columns=['epsilon', 'gamma', 'lower_bound', 'upper_bound','int_len'])

result

,epsilon,gamma,lower_bound,upper_bound,int_len
0,0.500,-0.00116,-0.03240,0.03008,0.062475
1,0.100,0.02100,-0.76454,0.80654,1.571076
2,0.050,-0.01600,-3.14573,3.11373,6.259457
3,0.005,-1.20000,-311.54918,309.14918,620.698363


In [93]:
## 2.2 Gamma using represtation

epsilons = [0.5, 0.1, 0.005, 0.005]
nr_sim = 500000
nr_eps = len(epsilons)
deltas = np.zeros(nr_eps)
gammas = np.zeros((nr_eps, 5))

for i in range(nr_eps): 
    epsilon_i = epsilons[i]

    XT, WT = sim_XT_WT(nr_sim=nr_sim)

    in_id = np.logical_and(XT>=a, XT<=b) 
    W_terms = WT ** 2 / (sigma * T) - WT - 1 / sigma
    
    gamma = 1 / (sigma  * T) * np.mean(in_id * W_terms)
    
    var_n = np.mean( ((in_id * W_terms) / (sigma  * T)) ** 2) - gamma ** 2
    half_int = ninty5_quantile * var_n ** 0.5 / nr_sim ** 0.5
    
    conf_interval = np.round([gamma - half_int,  gamma + half_int], 5)
    
    gammas[i, 0] = epsilon_i
    gammas[i, 1] = gamma  
    gammas[i, 2:4] = conf_interval
    gammas[i, 4] = half_int * 2 
    
    
import pandas as pd
result = pd.DataFrame(gammas,
                      columns=['epsilon', 'gamma', 'lower_bound', 'upper_bound','int_len'])

result  

,epsilon,gamma,lower_bound,upper_bound,int_len
0,0.500,-2.518004,-2.53418,-2.50183,0.032346
1,0.100,-2.522887,-2.53907,-2.50670,0.032372
2,0.005,-2.509980,-2.52613,-2.49383,0.032304
3,0.005,-2.507889,-2.52403,-2.49174,0.032291


In [79]:
# some draft code
# # # 

# # delta_plus

# # Compute the European call via Monte Carlo Method Based on Black Scholes model
# def sim_XT_WT_pertubation(epsilon, T=1, X0=100, r=0, sigma=0.25, nr_sim=10000):  
#     # this function generates the simulated XT and WT at the end of the valuation period 
#     WT = T ** 0.5 * np.random.randn(nr_sim, 1)
#     XT = (X0 + epsilon) * np.exp((r - sigma ** 2 / 2) * T + sigma * WT) 
#     return XT, WT


# XTplus, WTp = sim_XT_WT_pertubation(epsilon=0.05, nr_sim=100000)
# XTminus, WTm = sim_XT_WT_pertubation(epsilon=-0.05, nr_sim=100000)


# in_id = np.logical_and(XTplus>=a, XTplus<=b)
# plus_value = np.sum(in_id) /len(in_id)


# in_id = np.logical_and(XTminus>=a, XTminus<=b)
# minus_value = np.sum(in_id) /len(in_id)

# print(plus_value, minus_value)